In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import uniform

In [2]:
data = pickle.load(open("/content/drive/My Drive/Anadi_Basak_with_vocab/data/vocab_train.pkl", "rb"))
X_pred = (pd.read_pickle("/content/drive/My Drive/Anadi_Basak_with_vocab/data/vocab_gs_test.pkl"))

gc.collect()

25

In [0]:
# print(data.shape)
# print(X_pred.shape)
# data.loc[:,200].value_counts()

In [0]:
# 89704/(89704+524186)

In [3]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

target = data.loc[:,200]      
data.drop(columns = [200], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [4]:
y_train.value_counts()

0    366593
1     63130
Name: 200, dtype: int64

In [0]:
for i in range(3):
  X_train = X_train.append(X_train[y_train == 1])
  y_train = y_train.append(y_train[y_train == 1])

In [6]:
X_train.shape

(871633, 200)

In [7]:
model = XGBClassifier(tree_menthod = 'gpu_hist')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, tree_menthod='gpu_hist', verbosity=1)

In [8]:
pred = model.predict(X_test)
f1_score(pred, y_test)

0.3647108201124822

In [9]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []
acc = []
pred = (model.predict_proba(X_test))[:,1]

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cutoff, y=f1,
                    mode='lines',
                    name='f1 score'))
fig.add_trace(go.Scatter(x=cutoff, y=acc,
                    mode='lines',
                    name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [13]:
X_pred.shape

(8916, 200)

In [17]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Anadi_Basak_with_vocab/data/gs_sent.pickle")
GS_sent.columns = ["Sents", "doc_id"]

predictions = model.predict(X_pred)
probability_of_predictions = model.predict_proba(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1][:,1]
result['doc_id'] = result['doc_id']

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

,Sents,doc_id,probabilities
0,three civil appeals stemming three revision pe...,0.0,0.606592
1,high court deciding various issues remanded ca...,0.0,0.659692
2,shri achutananda purohit appellant intermediar...,0.0,0.518943
3,estate vested state april force act crucial qu...,0.0,0.529094
6,shri purohit appellant advocate profession yea...,0.0,0.654861
...,...,...,...
8900,thus manner doubt orders proceedings judicial ...,49.0,0.547629
8905,without entering niceties technicality subject...,49.0,0.776343
8908,exercise supervisory jurisdiction high court m...,49.0,0.556345
8909,appropriate cases high court exercising superv...,49.0,0.534536


In [0]:
result.to_csv("/content/drive/My Drive/BERT_Anadi/result_tfidf_vocab.csv")

# Model 1

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow as tf

x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)
print("Normalization Complete!")

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
# model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.4))
# model.add(tf.keras.layers.Dense(384, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.4))
# model.add(tf.keras.layers.Dense(384, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.4))
# model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.4))
# model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.4))
# model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.4))
# model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))   # hidden layer 1
# model.add(Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
print("Model Ready")

model.fit(x_train, np.array(y_train), epochs=10, validation_data = (x_test, np.array(y_test)))

Normalization Complete!
Model Ready
Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

19348/19348 [==============================] - 72s 4ms/step - loss: 0.5586 - accuracy: 0.7006 - val_loss: 0.5088 - val_accuracy: 0.7394
Epoch 2/10
19348/19348 [==============================] - 73s 4ms/step - loss: 0.5438 - accuracy: 0.7081 - val_loss: 0.5086 - val_accuracy: 0.7329
Epoch 3/10
19348/19348 [==============================] - 74s 4ms/step - loss: 0.5355 - accuracy: 0.7109 - val_loss: 0.5011 - val_accuracy: 0.7295
Epoch 4/10
19348/19348 [=======

In [0]:
cutoff = np.arange(0.01, 0.99, 0.01)
f1 = []
acc = []
pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=cutoff, y=f1,
                    mode='lines',
                    name='f1 score'))
fig.add_trace(go.Scatter(x=cutoff, y=acc,
                    mode='lines',
                    name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [0]:
max_cutoff

array([0.23])

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Kowshik_Srijan_NER_without_vocab/Data/GS_sent_500.pickle")
GS_sent.columns = ["Sents", "doc_id"]
predictions = (model.predict(X_pred))
probability_of_predictions = model.predict(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions >= max_cutoff])
result['probabilities'] = probability_of_predictions[predictions >= max_cutoff]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

IndexError: ignored